### Calculate edit distance between two sequences using dynamic programming
seq1: alpha * C (...BSDFDC)  
seq2: beta * A (...GEFDSA)  
*note: the alpha and belta is the prefix of the two sequences without the last base*

**edist(alpha * C, beta * A) = min(edist(alpha, beta) + 1, edist(alpha * C, beta) + 1, edist(alpha, beta * A) + 1)**

The edit distance of seq1 and seq2 is the **minimum** of:
1. edit distance of the prefix of the seq1 and seq2 and do a **substitution**  
2. edit distance of seq1 and the prefix of seq2 and **insert** A at the end of seq1  
2. edit distance of seq2 and the prefix of seq1 and **insert** C at the end of seq2  

The principle can be generalized into:  
  
seq1: alpha * X  
seq2: beta * Y  
  
**edist(alpha * X, belta * Y) = min(edist(alpha, belta) + delta(X, Y), edist(alpha * X, belta) + 1, edist(alpha, belta * Y) + 1)  
where delta(X, Y) = 0 if X = Y, or 1 otherwise**

In [10]:
def editdis(t, p):
    D = []
    for i in range(len(p) + 1):
        D.append([0] * (len(t) + 1))
    
    for i in range(len(p) + 1):
        for j in range(len(t) + 1):
            if i == 0:
                D[i][j] = j
            elif j == 0:
                D[i][j] = i
                
    for i in range(1, len(p) + 1):
        for j in range(1, len(t) + 1):       
            x = D[i-1][j] + 1
            y = D[i][j-1] + 1
            if p[i-1] == t[j-1]:
                z = D[i-1][j-1]
            else:
                z = D[i-1][j-1] + 1
            D[i][j] = min(x, y, z)
    return D[-1][-1]

In [12]:
#    *substitution of 's' with 'S'
#         *insert a ' ' in y
#              *insert a 'r' in x  
x = 'shake spea'
y = 'Shakespear'
editdis(x, y)

3

### Edit distance for approximate matching
Using the characters of T as columns and the characters of P as rows of the matrix. Because P can be matched at any postion in T, the first row of the matrix is 0. The minimal value in the bottom row is the edit distance of the closest match between P and T

The computation time of dynamic programming is proportional to the number of characters of P times the number of characters of T, while the computation time of booyer-moore is proportional to the length of T, and more skips will be available if there is longer P. However, booyer-moore can only do exact matching.

### Global alignment   vs. Local alignment 
Differences are:
1. Global alignment is **end-to-end** alignment that is made to the **entire** string, while local alignment finds the **substrings** of the sequences with highest similiarities **without considering the rest of sequences.**    
2. Global alignment compares **homologous sequences such same genes or proteins**, while local alignment is suitable for aligning more **divergent sequences or distantly related sequences**.  
3. Dynamic programming algorithms Needleman-Wunsch and Smith-Waterman are appropriate for global alignment and local alignment respectively. The score tracking starts from **bottom right corner** for Needleman-Wunsch while starts **position with highest score** for local alignment.  

find the most similar substring pairs of substring of x and y.

In [7]:
alphabet = ['A', 'C', 'T', 'G']
score = [[0, 4, 2, 4, 8],
         [4, 0, 4, 2, 8],  
         [2, 4, 0, 4, 8],  
         [4, 2, 4, 0, 8],
         [8, 8, 8, 8, 8]]
alphabet.index('C')

1

In [28]:
def globalAlignment(seq1, seq2):
    D = []
    for i in range(len(seq1) + 1):        #note: seq1 with become number of rows 
        D.append([0] * (len(seq2) + 1))   #note: seq2 with become number of columns
    
    for i in range(1, len(seq1) + 1):
        D[i][0] = D[i-1][0] + score[alphabet.index(seq1[i-1])][-1]
    for i in range(1, len(seq2) + 1):
        D[0][i] = D[0][i-1] + score[-1][alphabet.index(seq2[i-1])]
                
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):       
            horScore = D[i-1][j] + score[alphabet.index(seq1[i-1])][-1]
            verScore = D[i][j-1] + score[-1][alphabet.index(seq2[j-1])]
            if seq1[i-1] == seq2[j-1]:
                diagScore = D[i-1][j-1]
            else:
                diagScore = D[i-1][j-1] + score[alphabet.index(seq1[i-1])][alphabet.index(seq2[j-1])]
            D[i][j] = min(horScore, verScore, diagScore)
    return D[-1][-1]

In [29]:
# 10 * 11
x = 'TATGTCATGC'
y = 'TATGGCAGC'
print(globalAlignment(x,y))


12


Indexing works like a filter for exact match problems but doesn't deal very well with mismatches and gaps. Dynamic programming, on the the hand, is suitable for finding mismatches but is very slow for long reference sequences such as human DNA.  
In order to achieve efficient approximate matching, the combination of indexing and dynamic programming is a solution.

### Overlap graphs

In [50]:
def overlap(a, b, min_op = 3):
    lg = min(len(a), len(b))
    lap = []
    while min_op < lg:
        a_s = a[-min_op:]
        b_p = b[:min_op]
        if a_s == b_p:
            lap.append(min_op)
        min_op += 1
    return lap


In [53]:
overlap('TTACGTAGC', 'CGTAGCTGC')

[6]

In [61]:
test1 = 'TTACGTAGC'
test2 = 'CGTAGCTGC'
test1.find('TAG', 6) # return -1
test1.find('TAG', 3) # return 5, same as test1.find('TAG')

5

In [63]:
test1.startswith('TTA')

True

In [83]:
def overlap2(seq1, seq2, min_op = 3):
    start = 0
    while True:
        # find the position "start" in seq1 where prefix of seq2 matches 
        # substrings of a starting at that postion
        start = seq1.find(seq2[:min_op], start)
        if start == -1:
            return 0
        # verify the rest of string after the postion "start" matches the prefix of seq2, which means 
        # the suffix of seq1 matches the prefix of seq2
        if seq2.startswith(seq1[start:]):
            return len(seq1) - start
        start += 1

In [84]:
reslt = overlap2('TTACGT', 'CGTGTGC')
print(reslt)

3


In [85]:
from itertools import permutations
list(permutations([1,2, 3], 2))

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

In [86]:
def naive_overlap_map(reads, k):
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap2(a, b, min_op=k)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

In [87]:
reads = ['ACGGATC', 'GATCAAGT', 'TTCACGGA']
print(naive_overlap_map(reads, 3))

{('ACGGATC', 'GATCAAGT'): 4, ('TTCACGGA', 'ACGGATC'): 5}
